# 최종 예측 함수

In [709]:
# 상품군별 모델
model = [BaggingRegressor(LGBMRegressor(objective='regression',num_leaves=20,
                              learning_rate=0.01, n_estimators=900,
                              max_bin = 150, bagging_fraction = 0.9,
                              bagging_freq = 5, feature_fraction = 0.3,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf = 6, min_sum_hessian_in_leaf = 11),random_state = 1111) for i in range(11)]

In [710]:
class BigPython_Model:
    def __init__(self, model):
        # 상품군별 필요없는 칼럼(제외할 칼럼)
        self.del_features = {"이미용": ['실제_최고기온', '실제_최저기온', '실제_강수량', '실제_평균풍속'],
                     "침구": ['top_real_weather'],
                     '의류': ['분류'],
                     '가전': ['예보_최고기온', '예보_최저기온', '예보_강수확률', '예보_강수량', '예보_풍속'],
                     '잡화': ['top_real_weather'],
                     '가구': ['예보_최고기온', '예보_최저기온', '예보_강수확률', '예보_강수량', '예보_풍속'],
                     '주방': ['top_cat'],
                     '건강기능': ['top_real_weather'],
                     '속옷': [],
                     '생활용품': ['top_code'],
                     '농수축': ['top_cat']}
        
        # train 시와 동일하게 전처리하기 위한 딕셔너리
        self.product_home = {'f1': 31,'g11': 41,'g9': 46,'b3': 4,'j4': 76,'b6': 7,'b5': 6,'g2': 42,'f3': 34,'d33': 22,'d5': 23,
 'b2': 3,'b1': 2,'f5': 36,'f7': 38,'f2': 33,'c3': 10,'g4': 44,'j6': 77,'k2': 82,'j13': 68,'f10': 32,'j3': 75,'d26': 16,
 'c1': 8,'a1': 0,'h3': 53,'i4': 61,'j2': 73,'e18': 28,'j12': 67,'j14': 69,'d15': 13,'i3': 60,'f4': 35,'h11': 47,'g3': 43,
 'd1': 11,'h14': 49,'e1': 27,'j15': 70,'a2': 1,'i1': 56,'i29': 59,'h2': 52,'f6': 37,'d14': 12,'j1': 64,'d29': 19,
 'c2': 9,'f8': 39, 'j8': 79,'j10': 65,'d28': 18,'d9': 26,'i2': 58,'f9': 40,'e4': 29,'j20': 74,'g5': 45,'j9': 80,'j17': 72,'h4': 54,
 'k1': 81,'e8': 30,'j16': 71,'i7': 63,'j7': 78,'i10': 57,'d3': 20,'h13': 48,'h16': 51,'j11': 66,'d7': 25,
 'h9': 55,'d27': 17,'d6': 24,'i6': 62,'b4': 5,'h15': 50,'d30': 21,'d21': 14,'d24': 15}
        
        # 상품군별 모델
        self.models=model
        
        # 코로나 effect
        self.corona={'의류':1.192,'속옷':1,'이미용':0.996,'주방':1,'농수축':1.236,'가전':1.188,'생활용품':1.292,'가구':1.328,'잡화':0.996,'건강기능':1.62,'침구':1}
        
    def Split_Category(self, data, train=True): # 카테고리별로 나눔
        
        # 범주형변수 LabelEncoding
        data["분류"] = data["분류"].apply(lambda x: self.product_home[x])
        
        # 상품군별로 데이터 반환
        for c,x in data.groupby(["상품군"]):
            del x["상품군"]
            
            if train: # 훈련시
                # 카테고리 이름, X, y 반환
                yield c, x.drop(["주문량"], axis=1), x["주문량"]
                
            else:  # 예측시
                # 카테고리 이름, X 반환
                yield c, x
    
    def Train(self, data,train=True): # MAPE 결과를 출력하는 함수
        # 상품군별 데이터를 받아옴
        for idx, (c, X, y) in enumerate(self.Split_Category(data.copy())):
            # 훈련
            self.models[idx].fit(X,y)
    

    def Predict(self, data, train=False, corona = True):
        # 예측값을 넣을 배열
        self.predicted = np.zeros(data.shape[0])
        # 상품군별 데이터를 받아옴
        for idx, (c, X) in enumerate(self.Split_Category(data.copy(),train=train)):
            if c=="무형":
                pass # 무형이면 예측으로 0이기 때문에 pass
            
            # 예측
            else:
                # 해당 상품군의 index를 받아옴
                predicted_idx = X.index
                if corona: # corona effect 가 있었을 시기 -> corona effect 적용
                    
                    # 해당 인덱스에 예측값 저장
                    self.predicted[X.index] = self.models[idx].predict(X)*self.corona[c]
                else:
                    # 해당 인덱스에 예측값 저장
                    self.predicted[X.index] = self.models[idx].predict(X)
                
        return self.predicted
        
        

In [ ]:
bpm1 = BigPython_Model(model)
bpm1.Train(raw)

In [702]:
bpm1.Predict(raw, train=False, corona=True)

array([457.04934258, 457.04934258, 457.04934258, ..., 103.03239796,
       103.03239796, 103.03239796])